# Student Loan Risk with Deep Learning

In [53]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path
from google.colab import drive

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [ ]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [ ]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [ ]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [ ]:
# Define the target set y using the credit_ranking column
y = loans_df['credit_ranking']

# Display a sample of y
y.head()

0    0
1    0
2    0
3    1
4    0
Name: credit_ranking, dtype: int64

In [ ]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop('credit_ranking', axis=1)

# Review the features DataFrame
X.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [ ]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [ ]:
# Create a StandardScaler instance
X_scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [ ]:
# Define the the number of inputs (features) to the model
inputs = len(X.columns)

# Review the number of features
inputs

11

In [ ]:
# Define the number of hidden nodes for the first hidden layer
layer_1_nodes = 22

# Define the number of hidden nodes for the second hidden layer
layer_2_nodes = 22

# Define the number of neurons in the output layer
output_neurons = 1

In [ ]:
# Create the Sequential model instance
nn_model = tf.keras.models.Sequential()

# Add the first hidden layer
nn_model.add(tf.keras.layers.Dense(units=layer_1_nodes, activation='relu', input_dim=inputs))

# Add the second hidden layer
nn_model.add(tf.keras.layers.Dense(units=layer_2_nodes, activation='relu'))

# Add the output layer to the model specifying the number of output neurons and activation function
nn_model.add(tf.keras.layers.Dense(units=output_neurons, activation='sigmoid'))

In [ ]:
# Display the Sequential model summary
nn_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 22)                264       
                                                                 
 dense_8 (Dense)             (None, 22)                506       
                                                                 
 dense_9 (Dense)             (None, 1)                 23        
                                                                 
Total params: 793 (3.10 KB)
Trainable params: 793 (3.10 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [ ]:
# Compile the Sequential model
nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])

In [ ]:
# Fit the model using 50 epochs and the training data
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
38/38 [==============================] - 1s 2ms/step - loss: 0.6711 - accuracy: 0.5888
Epoch 2/50
38/38 [==============================] - 0s 2ms/step - loss: 0.6317 - accuracy: 0.7148
Epoch 3/50
38/38 [==============================] - 0s 3ms/step - loss: 0.5846 - accuracy: 0.7465
Epoch 4/50
38/38 [==============================] - 0s 2ms/step - loss: 0.5450 - accuracy: 0.7456
Epoch 5/50
38/38 [==============================] - 0s 2ms/step - loss: 0.5235 - accuracy: 0.7598
Epoch 6/50
38/38 [==============================] - 0s 2ms/step - loss: 0.5123 - accuracy: 0.7640
Epoch 7/50
38/38 [==============================] - 0s 2ms/step - loss: 0.5059 - accuracy: 0.7623
Epoch 8/50
38/38 [==============================] - 0s 2ms/step - loss: 0.5005 - accuracy: 0.7615
Epoch 9/50
38/38 [==============================] - 0s 2ms/step - loss: 0.4959 - accuracy: 0.7656
Epoch 10/50
38/38 [==============================] - 0s 2ms/step - loss: 0.4909 - accuracy: 0.7673
Epoch 11/50
38/38 [

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [ ]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

13/13 - 0s - loss: 0.5294 - accuracy: 0.7550 - 166ms/epoch - 13ms/step
Loss: 0.5294389128684998, Accuracy: 0.7549999952316284


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [ ]:
# Set the model's file path
file_path = Path('student_loans.keras')

# Export your model to a keras file
nn_model.save(file_path)

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [ ]:
# Set the model's file path
file_path = Path('student_loans.keras')

# Load the model to a new object
nn_imported = tf.keras.models.load_model(file_path)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [ ]:
# Make predictions with the test data
predictions = nn_model.predict(X_test_scaled, verbose=2)

# Display a sample of the predictions
predictions[:10]

13/13 - 0s - 62ms/epoch - 5ms/step


array([[0.2382451 ],
       [0.34482092],
       [0.7530564 ],
       [0.709387  ],
       [0.9878242 ],
       [0.91057193],
       [0.97081137],
       [0.02222132],
       [0.8186674 ],
       [0.31285158]], dtype=float32)

In [ ]:
# Save the predictions to a DataFrame and round the predictions to binary results
predictions_df = pd.DataFrame(columns=["predictions"], data=predictions)
predictions_df["predictions"] = round(predictions_df['predictions'], 0)
predictions_df

,predictions
0,0.0
1,0.0
2,1.0
3,1.0
4,1.0
...,...
395,1.0
396,0.0
397,1.0
398,0.0


### Step 4: Display a classification report with the y test data and predictions

In [ ]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, predictions_df["predictions"].values))

              precision    recall  f1-score   support

           0       0.72      0.78      0.75       188
           1       0.79      0.73      0.76       212

    accuracy                           0.76       400
   macro avg       0.76      0.76      0.75       400
weighted avg       0.76      0.76      0.76       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

## **Answers to the above questions can be found below.**

**1. I would need to collect the types of loans other students have chosen with similar data values to the ones given in this example. This data would be relevant because it would show what the preferred loans were amongst their peers.**


**2. My model would be using collaborative filtering. The reason the data I have chosen is suitable for collaborative filtering is because it is data that is being obtained from the students peers that they have common current loan/student data with.**


**3. Two real-world challenges I would face with this type of recommendation system would be privacy and trust and diversity of recommendations. These would be challenges because financials are known to be private entities for a person and that could infringe on their trust in the loan suppliers. Diversity of recommendations would be a problem because the system may only show a tunnel vision view of loans available to the person based on similar peers but not those that differ from them, thus, only giving them narrowed options/suggestions.**

